# 실험 5: K-ClassicBench 벤치마크 평가

**목표**: C3Bench를 참고하여 개발한 K-ClassicBench로 다양한 LLM 평가

## 평가 모델 종류

### 1. 비공개 API 모델
- GPT-4 Turbo
- GPT-3.5 Turbo
- Claude 3.5 Sonnet
- Claude 3 Opus

### 2. 오픈소스 모델
- Llama 3.1 (8B, 70B)
- Qwen 2.5 (7B, 14B, 72B)
- EXAONE 3.0 (7.8B)

### 3. 지도학습 모델
- GwenBert
- Tongu

## 평가 태스크

1. **Classification**: 문체 분류 (賦/詩/疑/義 등)
2. **Retrieval**: 출처 식별 (論語/孟子/大學/中庸)
3. **Punctuation**: 구두점 복원 (백문 → 구두점본)
4. **NLI**: 자연언어추론 (entailment/contradiction/neutral)
5. **Translation**: 번역 (한문↔한글↔영문)

## 1. 환경 설정

In [ ]:
import sys
import os
import json
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List

# 한글 폰트 설정
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

# 경로 설정
BENCHMARK_PATH = '/Users/songhune/Workspace/korean_eda/benchmark/k_classic_bench/k_classic_bench_full.json'
RESULTS_DIR = '/Users/songhune/Workspace/korean_eda/benchmark/results'

# 결과 디렉토리 생성
os.makedirs(RESULTS_DIR, exist_ok=True)

print("✅ 환경 설정 완료")

## 2. 벤치마크 데이터 확인

In [ ]:
# 벤치마크 로드
with open(BENCHMARK_PATH, 'r', encoding='utf-8') as f:
    benchmark = json.load(f)

print(f"📊 {benchmark['benchmark_info']['name']}")
print(f"버전: {benchmark['benchmark_info']['version']}")
print(f"총 항목 수: {benchmark['benchmark_info']['total_size']:,}개")
print(f"\n태스크:")

for task_name, task_data in benchmark['tasks'].items():
    print(f"  - {task_name}: {task_data['size']:,}개 ({task_data['metric']})")

In [ ]:
# 각 태스크 예시 확인
for task_name, task_data in benchmark['tasks'].items():
    print(f"\n{'='*70}")
    print(f"[{task_name.upper()}] 예시")
    print(f"{'='*70}")
    
    example = task_data['data'][0]
    
    if task_name == 'classification':
        print(f"입력: {example['input']}")
        print(f"레이블: {example['label']}")
    
    elif task_name == 'retrieval':
        print(f"입력: {example['input'][:100]}...")
        print(f"정답: {example['answer']}")
    
    elif task_name == 'punctuation':
        print(f"입력 (백문): {example['input'][:80]}...")
        print(f"정답 (구두점): {example['answer'][:80]}...")
    
    elif task_name == 'nli':
        print(f"전제: {example['premise'][:80]}...")
        print(f"가설: {example['hypothesis'][:80]}...")
        print(f"레이블: {example['label']}")
    
    elif task_name == 'translation':
        print(f"원문 ({example['source_lang']}): {example['source_text'][:80]}...")
        print(f"번역 ({example['target_lang']}): {example['target_text'][:80]}...")

## 3. 평가 실행

### 3.1 API 모델 평가 (GPT-4, Claude)

In [ ]:
# API 키 설정 (환경 변수 또는 직접 입력)
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')  # 또는 직접 입력
ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY')  # 또는 직접 입력

# API 키 확인
print(f"OpenAI API Key: {'설정됨' if OPENAI_API_KEY else '미설정'}")
print(f"Anthropic API Key: {'설정됨' if ANTHROPIC_API_KEY else '미설정'}")

In [ ]:
# GPT-4 평가 (샘플 테스트: 각 태스크당 10개)
!python exp5_benchmark_evaluation.py \
    --model-type api \
    --model-name gpt-4-turbo \
    --max-samples 10 \
    --api-key $OPENAI_API_KEY

In [ ]:
# Claude 3.5 Sonnet 평가 (샘플 테스트)
!python exp5_benchmark_evaluation.py \
    --model-type api \
    --model-name claude-3-5-sonnet-20241022 \
    --max-samples 10 \
    --api-key $ANTHROPIC_API_KEY

### 3.2 오픈소스 모델 평가

In [ ]:
# Llama 3.1 8B 평가 (샘플 테스트)
# 주의: GPU 메모리 필요
!python exp5_benchmark_evaluation.py \
    --model-type opensource \
    --model-name meta-llama/Llama-3.1-8B-Instruct \
    --max-samples 10

In [ ]:
# Qwen 2.5 7B 평가 (샘플 테스트)
!python exp5_benchmark_evaluation.py \
    --model-type opensource \
    --model-name Qwen/Qwen2.5-7B-Instruct \
    --max-samples 10

### 3.3 지도학습 모델 평가 (GwenBert, Tongu)

**주의**: 이 부분은 모델별 구현이 필요합니다.

In [ ]:
# Tongu 모델 평가
# TODO: Tongu 모델 경로 지정 및 래퍼 구현 필요
# !python exp5_benchmark_evaluation.py \
#     --model-type supervised \
#     --model-name tongu \
#     --max-samples 10

## 4. 결과 분석

In [ ]:
# 결과 파일 로드
result_files = list(Path(RESULTS_DIR).glob('summary_*.csv'))

print(f"📁 발견된 결과 파일: {len(result_files)}개")
for f in result_files:
    print(f"  - {f.name}")

In [ ]:
# 모든 결과 병합
all_results = []

for file in result_files:
    df = pd.read_csv(file)
    all_results.append(df)

if all_results:
    results_df = pd.concat(all_results, ignore_index=True)
    print(f"✅ 총 {len(results_df)} 개의 결과 로드됨")
    display(results_df.head())
else:
    print("⚠️  결과 파일이 없습니다. 먼저 평가를 실행하세요.")

### 4.1 태스크별 성능 비교

In [ ]:
if all_results:
    # 태스크별 주요 메트릭 추출
    metric_map = {
        'classification': 'accuracy',
        'retrieval': 'accuracy',
        'punctuation': 'rougeL_f1',
        'nli': 'accuracy',
        'translation': 'bleu'
    }
    
    # 피벗 테이블 생성
    pivot_data = []
    
    for task, metric in metric_map.items():
        task_data = results_df[results_df['task'] == task]
        for _, row in task_data.iterrows():
            pivot_data.append({
                'model': row['model'],
                'task': task,
                'score': row.get(metric, 0)
            })
    
    pivot_df = pd.DataFrame(pivot_data)
    pivot_table = pivot_df.pivot(index='model', columns='task', values='score')
    
    print("\n📊 태스크별 성능 (주요 메트릭)")
    display(pivot_table)

### 4.2 시각화

In [ ]:
if all_results:
    # 태스크별 성능 비교 그래프
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    fig.suptitle('K-ClassicBench: 태스크별 모델 성능 비교', fontsize=16, fontweight='bold')
    
    tasks = list(metric_map.keys())
    
    for idx, task in enumerate(tasks):
        ax = axes[idx // 3, idx % 3]
        
        task_df = pivot_df[pivot_df['task'] == task]
        
        if not task_df.empty:
            task_df = task_df.sort_values('score', ascending=True)
            
            ax.barh(task_df['model'], task_df['score'], color='skyblue')
            ax.set_xlabel('Score')
            ax.set_title(f'{task.upper()} ({metric_map[task]})')
            ax.set_xlim(0, 1.0)
            ax.grid(axis='x', alpha=0.3)
    
    # 빈 subplot 제거
    if len(tasks) < 6:
        for idx in range(len(tasks), 6):
            fig.delaxes(axes[idx // 3, idx % 3])
    
    plt.tight_layout()
    plt.savefig(f'{RESULTS_DIR}/performance_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
if all_results:
    # 히트맵
    plt.figure(figsize=(10, 6))
    sns.heatmap(pivot_table, annot=True, fmt='.3f', cmap='YlOrRd', cbar_kws={'label': 'Score'})
    plt.title('모델별 태스크 성능 히트맵', fontsize=14, fontweight='bold')
    plt.xlabel('태스크')
    plt.ylabel('모델')
    plt.tight_layout()
    plt.savefig(f'{RESULTS_DIR}/heatmap.png', dpi=300, bbox_inches='tight')
    plt.show()

### 4.3 모델별 종합 점수

In [ ]:
if all_results:
    # 평균 점수 계산
    model_avg = pivot_table.mean(axis=1).sort_values(ascending=False)
    
    print("\n🏆 모델별 평균 성능 (전체 태스크)")
    print("="*50)
    for idx, (model, score) in enumerate(model_avg.items(), 1):
        print(f"{idx}. {model}: {score:.4f}")
    
    # 그래프
    plt.figure(figsize=(10, 6))
    model_avg.plot(kind='barh', color='coral')
    plt.xlabel('평균 점수')
    plt.title('모델별 평균 성능', fontsize=14, fontweight='bold')
    plt.grid(axis='x', alpha=0.3)
    plt.tight_layout()
    plt.savefig(f'{RESULTS_DIR}/overall_performance.png', dpi=300, bbox_inches='tight')
    plt.show()

## 5. 상세 분석

### 5.1 Classification: 문체별 정확도

In [ ]:
# Classification 상세 결과 로드
# TODO: 각 모델의 JSON 결과 파일에서 예측값 추출하여 분석

### 5.2 오류 분석

In [ ]:
# 오답 케이스 분석
# TODO: JSON 결과에서 예측 실패 케이스 추출 및 분석

## 6. 결론 및 인사이트

### 주요 발견사항

1. **모델별 성능**
   - 비공개 API 모델 (GPT-4, Claude)의 전반적인 우수성
   - 오픈소스 모델의 태스크별 편차
   - 지도학습 모델의 특정 태스크 강점

2. **태스크별 난이도**
   - 가장 쉬운 태스크: [분석 필요]
   - 가장 어려운 태스크: [분석 필요]

3. **개선 방향**
   - 프롬프트 엔지니어링
   - Few-shot learning 적용
   - Fine-tuning 가능성

### C3Bench와의 비교

- [분석 필요]

### 향후 연구 방향

1. 더 많은 모델 평가
2. Few-shot 성능 비교
3. 도메인 특화 Fine-tuning
4. 벤치마크 확장 (더 많은 태스크, 데이터)

## 7. 부록: 전체 평가 실행 스크립트

### 배치 평가 스크립트

In [ ]:
# 여러 모델을 순차적으로 평가하는 스크립트
api_models = [
    ('gpt-4-turbo', 'openai'),
    ('gpt-3.5-turbo', 'openai'),
    ('claude-3-5-sonnet-20241022', 'anthropic'),
]

opensource_models = [
    'meta-llama/Llama-3.1-8B-Instruct',
    'Qwen/Qwen2.5-7B-Instruct',
    'LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct',
]

# API 모델 평가
for model_name, provider in api_models:
    print(f"\n{'='*70}")
    print(f"평가 중: {model_name}")
    print(f"{'='*70}")
    
    api_key = OPENAI_API_KEY if provider == 'openai' else ANTHROPIC_API_KEY
    
    !python exp5_benchmark_evaluation.py \
        --model-type api \
        --model-name {model_name} \
        --api-key {api_key}

# 오픈소스 모델 평가
for model_name in opensource_models:
    print(f"\n{'='*70}")
    print(f"평가 중: {model_name}")
    print(f"{'='*70}")
    
    !python exp5_benchmark_evaluation.py \
        --model-type opensource \
        --model-name {model_name}